# 數據整合平台 v15 - 智慧更新版

歡迎使用！這是一個具備智慧更新功能的自動化數據處理工具。

---
### **首次使用引導**
**重要：** 這是一個公版的「樣板筆記本」。為了保存您自己的設定和執行結果，請務必先執行以下操作：
1. 在 Colab 選單列點擊 **「檔案」(File)**。
2. 選擇 **「在雲端硬碟中儲存複本」(Save a copy in Drive)**。
3. Colab 會開啟一個您專屬的「個人版筆記本」，請在您個人的版本上進行所有操作。

---
### **版本管理說明**
本筆記本具備「非破壞性」的智慧更新機制：
* **自動檢查**：當您執行時，它會自動檢查 GitHub 上是否有新版本。
* **詢問意願**：如果偵測到新版，它會詢問您的意願，您可以選擇繼續使用舊版，或前往取得新版。
* **保留舊檔**：當您決定取得新版並存檔時，您目前的版本會被完整保留，讓您可以隨時比較或還原。

---
### **更新日誌**
* **v2.0.0 (2025-06-19):**
    * 導入時間戳記智慧更新機制。
    * 新增輸出檔名與來源檔名之參數化設定。
* **v1.0.0 (2025-06-18):**
    * 建立初始整合式啟動腳本。

In [ ]:
#@markdown ## 智慧更新檢查
#@markdown 執行此儲存格將首先檢查是否有新版本。
import requests
import json
from datetime import datetime

# --- 內嵌時間戳記 (由Jules在生成時填入) ---
NOTEBOOK_TIMESTAMP = "2025-06-19T05:30:47Z"

def check_for_updates():
    try:
        print("⏳ 正在檢查是否有新版本...")
        api_url = "https://api.github.com/repos/hsp1234-web/taifexd-date/commits?path=data_pipeline_v15/run_v15.ipynb&page=1&per_page=1"
        response = requests.get(api_url)
        response.raise_for_status()
        latest_commit_date_str = response.json()[0]['commit']['committer']['date']
        latest_timestamp = datetime.fromisoformat(latest_commit_date_str.replace('Z', '+00:00'))
        current_timestamp = datetime.fromisoformat(NOTEBOOK_TIMESTAMP.replace('Z', '+00:00'))

        if latest_timestamp > current_timestamp:
            print(f"⚠️ 偵測到新版本！")
            print(f"- 您目前版本時間：{current_timestamp.strftime('%Y-%m-%d')}")
            print(f"- 最新版本時間：{latest_timestamp.strftime('%Y-%m-%d')}")
            
            choice = input("請問您要如何處理？ (1) 繼續使用我目前的舊版本 (2) 前往查看並取得新版本\n請輸入您的選擇 (1 或 2): ")
            if choice == '2':
                print("\n請點擊以下連結開啟新版本，並依循首次使用引導，將其另存為新的複本：")
                print("https://colab.research.google.com/github/hsp1234-web/taifexd-date/blob/main/data_pipeline_v15/run_v15.ipynb")
                return False # 中斷後續執行
        else:
            print("✅ 您目前使用的已是最新版本。")
    except Exception as e:
        print(f"- 更新檢查失敗：{e}，將繼續執行目前版本。")
    return True # 繼續執行

if check_for_updates():
    #@markdown ---
    #@markdown ## 執行參數設定
    #@markdown 請在下方輸入您的執行參數，若不修改則使用預設值。
    project_folder = "MyTaifexDataProject" #@param {type:"string"}
    database_name = "processed_data.duckdb" #@param {type:"string"}
    log_name = "pipeline.log" #@param {type:"string"}
    target_zip_files = "" #@param {type:"string"}
    #@markdown *若要指定多個ZIP檔，請用逗號分隔，例如 `file1.zip,file2.zip`。留空則代表處理全部。*

    print("\n--- 參數確認 ---")
    print(f"專案資料夾: {project_folder}")
    print(f"輸出資料庫: {database_name}")
    print(f"輸出日誌檔: {log_name}")
    print(f"指定ZIP檔: {'全部' if not target_zip_files else target_zip_files}")
    print("--------------------")

    #@markdown ---
    #@markdown ## 執行主要程式
    import os

    # --- 掛載 Google Drive ---
    try:
        from google.colab import drive
        print("\n⏳ 正在請求掛載 Google Drive...")
        drive.mount('/content/drive', force_remount=True)
        print("✅ Google Drive 掛載成功！")
    except ImportError:
        print("非 Colab 環境，跳過掛載 Drive。")

    # --- 下載/更新專案 ---
    print("\n⏳ 正在準備專案程式碼...")
    if os.path.exists('/content/taifexd-date'):
        os.chdir('/content/taifexd-date')
        !git pull
    else:
        !git clone -q https://github.com/hsp1234-web/taifexd-date.git
    project_path = "/content/taifexd-date/data_pipeline_v15"
    os.chdir(project_path)
    print("✅ 專案程式碼準備完成！")

    # -------------------- 注入缺失的 import --------------------\n    print("執行前修補：為 pipeline_orchestrator.py 注入缺失的 import...")\n    file_to_patch = '/content/taifexd-date/data_pipeline_v15/src/data_pipeline_v15/pipeline_orchestrator.py'\n    try:\n        with open(file_to_patch, 'r+', encoding='utf-8') as f:\n            content = f.read()\n            if 'from pathlib import Path' not in content:\n                f.seek(0, 0)\n                f.write('from pathlib import Path\n' + content)\n                print("修補成功！")\n            else:\n                print("無需修補，import 已存在。")\n    except FileNotFoundError:\n        print(f"錯誤：找不到檔案 {file_to_patch}，請檢查路徑是否正確。")\n    # -----------------------------------------------------------\n\n    # --- 安裝相依套件 ---
    print("\n⏳ 正在安裝必要的 Python 套件...")
    !pip install -q pandas duckdb psutil pyarrow\n    print("✅ 套件安裝完成！")

    # --- 建構並執行指令 ---
    print("\n🚀 即將啟動數據整合平台...")
    command = (
        f"python main.py --project-folder-name='{project_folder}' "
        f" --database-name='{database_name}' "
        f" --log-name='{log_name}' "
        f" --zip-files='{target_zip_files}'"
    )
    print(f"執行指令: {command}")
    print("-" * 20)
    !{command}
    print("-" * 20)
    print("🎉 執行完畢！")